In [1]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y


######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - \ done

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - \ | / done

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - done

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - \ done

#

In [2]:
import sys
import os
from PIL import Image
import pandas as pd
from tqdm.auto import tqdm
import gc
import glob
import numpy as np

In [3]:
sub_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')

# Form study and image dataframes
sub_df['level'] = sub_df.id.map(lambda idx: idx[-5:])
study_df = sub_df[sub_df.level=='study'].rename({'id':'study_id'}, axis=1)
image_df = sub_df[sub_df.level=='image'].rename({'id':'image_id'}, axis=1)

dcm_path = glob.glob('/kaggle/input/siim-covid19-detection/test/**/*dcm', recursive=True)
test_meta = pd.DataFrame({'dcm_path':dcm_path})
test_meta['image_id'] = test_meta.dcm_path.map(lambda x: x.split('/')[-1].replace('.dcm', '')+'_image')
test_meta['study_id'] = test_meta.dcm_path.map(lambda x: x.split('/')[-3].replace('.dcm', '')+'_study')

study_df = study_df.merge(test_meta, on='study_id', how='left')
image_df = image_df.merge(test_meta, on='image_id', how='left')

# Remove duplicates study_ids from study_df
study_df.drop_duplicates(subset="study_id",keep='first', inplace=True)

In [4]:
fast_sub = False

if sub_df.shape[0] == 2477:
    fast_sub = True
    study_df = study_df.sample(2)
    image_df = image_df.sample(2)
    fast_df = image_df.copy()
    
    print("\nstudy_df")
    display(study_df.head(2))
    print("\nimage_df")
    display(image_df.head(2))
    print("\ntest_meta")
    display(test_meta.head(2))


study_df


,study_id,PredictionString,level,dcm_path,image_id
1046,ce2d48a26e5e_study,negative 1 0 0 1 1,study,/kaggle/input/siim-covid19-detection/test/ce2d...,a1883aeaa8f0_image
68,0ec0901a5d45_study,negative 1 0 0 1 1,study,/kaggle/input/siim-covid19-detection/test/0ec0...,4d7b06512cc1_image



image_df


,image_id,PredictionString,level,dcm_path,study_id
351,2a2f54eb48df_image,none 1 0 0 1 1,image,/kaggle/input/siim-covid19-detection/test/def2...,def217a20092_study
1206,27d3da7a8e3d_image,none 1 0 0 1 1,image,/kaggle/input/siim-covid19-detection/test/e5eb...,e5eb6f1abf7f_study



test_meta


,dcm_path,image_id,study_id
0,/kaggle/input/siim-covid19-detection/test/2fb1...,a29c5a68b07b_image,2fb11712bc93_study
1,/kaggle/input/siim-covid19-detection/test/19c6...,9850b5470fd6_image,19c66935e737_study


In [5]:
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

STUDY_DIMS = (768, 768)
IMAGE_DIMS = (768, 768)

study_dir = f'/kaggle/tmp/test/study/'
os.makedirs(study_dir, exist_ok=True)

image_dir = f'/kaggle/tmp/test/image_2class/'
os.makedirs(image_dir, exist_ok=True)

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    return im

for index, row in tqdm(study_df[['study_id', 'dcm_path']].iterrows(), total=study_df.shape[0]):
    # set keep_ratio=True to have original aspect ratio
    xray = read_xray(row['dcm_path'])
    im = resize(xray, size=STUDY_DIMS[0])
    im.save(os.path.join(study_dir, row['study_id']+'.png'))

image_df['dim0'] = -1
image_df['dim1'] = -1

for index, row in tqdm(image_df[['image_id', 'dcm_path', 'dim0', 'dim1']].iterrows(), total=image_df.shape[0]):
    # set keep_ratio=True to have original aspect ratio
    xray = read_xray(row['dcm_path'])
    im = resize(xray, size=IMAGE_DIMS[0])  
    im.save(os.path.join(image_dir, row['image_id']+'.png'))
    image_df.loc[image_df.image_id==row.image_id, 'dim0'] = xray.shape[0]
    image_df.loc[image_df.image_id==row.image_id, 'dim1'] = xray.shape[1]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
study_df['image_path'] = study_dir+study_df['study_id']+'.png'
image_df['image_path'] = image_dir+image_df['image_id']+'.png'

In [7]:
import tensorflow as tf
import tensorflow_hub as tfhub

MODEL_ARCH = 'efficientnetv2-l-21k-ft1k'
# Get the TensorFlow Hub model URL
hub_type = 'feature_vector' # ['classification', 'feature_vector']
MODEL_ARCH_PATH = f'/kaggle/input/efficientnetv2-tfhub-weight-files/tfhub_models/{MODEL_ARCH}/{hub_type}'

# Custom wrapper class to load the right pretrained weights explicitly from the local directory
class KerasLayerWrapper(tfhub.KerasLayer):
    def __init__(self, handle, **kwargs):
        handle = tfhub.KerasLayer(tfhub.load(MODEL_ARCH_PATH))
        super().__init__(handle, **kwargs)

# study predict

In [8]:
MODEL_PATH = '/kaggle/input/siim-effnetv2-keras-study-train-tpu-cv0-805'
test_paths = study_df.image_path.tolist()
BATCH_SIZE = 16

def build_decoder(with_labels=True, target_size=(300, 300), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img

    def decode_with_labels(path, label):
        return decode(path), label

    return decode_with_labels if with_labels else decode

def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img

    def augment_with_labels(img, label):
        return augment(img), label

    return augment_with_labels if with_labels else augment

def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)

    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)

    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)

    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)

    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)

    return dset

# strategy = auto_select_accelerator()
# BATCH_SIZE = strategy.num_replicas_in_sync * 16

label_cols = ['negative', 'typical', 'indeterminate', 'atypical']
study_df[label_cols] = 0

test_decoder = build_decoder(with_labels=False,
                             target_size=(STUDY_DIMS[0],
                                          STUDY_DIMS[0]), ext='png')
test_dataset = build_dataset(
    test_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False,
    decode_fn=test_decoder
)

with tf.device('/device:GPU:0'):
    models = []
    models0 = tf.keras.models.load_model(f'{MODEL_PATH}/model0.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models1 = tf.keras.models.load_model(f'{MODEL_PATH}/model1.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models2 = tf.keras.models.load_model(f'{MODEL_PATH}/model2.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models3 = tf.keras.models.load_model(f'{MODEL_PATH}/model3.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models4 = tf.keras.models.load_model(f'{MODEL_PATH}/model4.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models.append(models0)
    models.append(models1)
    models.append(models2)
    models.append(models3)
    models.append(models4)

study_df[label_cols] = sum([model.predict(test_dataset, verbose=1) for model in models]) / len(models)
study_df['PredictionString'] = study_df[label_cols].apply(lambda row: f'negative {row.negative} 0 0 1 1 typical {row.typical} 0 0 1 1 indeterminate {row.indeterminate} 0 0 1 1 atypical {row.atypical} 0 0 1 1', axis=1)

del models
del models0, models1, models2, models3, models4
del test_dataset, test_decoder
gc.collect()

1/1 [==============================] - 5s 5s/step


4747974

In [9]:
# # MODEL_PATH = '/kaggle/input/siimeffnetv2kerasstudy-train'
# MODEL_PATH = 'kaggle/input/siim-effnetv2-keras-study-train-tpu-cv0-805'
# test_paths = study_df.image_path.tolist()
# BATCH_SIZE = 16

# def build_decoder(with_labels=True, target_size=(300, 300), ext='jpg'):
#     def decode(path):
#         file_bytes = tf.io.read_file(path)
#         if ext == 'png':
#             img = tf.image.decode_png(file_bytes, channels=3)
#         elif ext in ['jpg', 'jpeg']:
#             img = tf.image.decode_jpeg(file_bytes, channels=3)
#         else:
#             raise ValueError("Image extension not supported")

#         img = tf.cast(img, tf.float32) / 255.0
#         img = tf.image.resize(img, target_size)

#         return img

#     def decode_with_labels(path, label):
#         return decode(path), label

#     return decode_with_labels if with_labels else decode

# def build_augmenter(with_labels=True):
#     def augment(img):
#         img = tf.image.random_flip_left_right(img)
#         img = tf.image.random_flip_up_down(img)
#         return img

#     def augment_with_labels(img, label):
#         return augment(img), label

#     return augment_with_labels if with_labels else augment

# def build_dataset(paths, labels=None, bsize=32, cache=True,
#                   decode_fn=None, augment_fn=None,
#                   augment=True, repeat=True, shuffle=1024, 
#                   cache_dir=""):
#     if cache_dir != "" and cache is True:
#         os.makedirs(cache_dir, exist_ok=True)

#     if decode_fn is None:
#         decode_fn = build_decoder(labels is not None)

#     if augment_fn is None:
#         augment_fn = build_augmenter(labels is not None)

#     AUTO = tf.data.experimental.AUTOTUNE
#     slices = paths if labels is None else (paths, labels)

#     dset = tf.data.Dataset.from_tensor_slices(slices)
#     dset = dset.map(decode_fn, num_parallel_calls=AUTO)
#     dset = dset.cache(cache_dir) if cache else dset
#     dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
#     dset = dset.repeat() if repeat else dset
#     dset = dset.shuffle(shuffle) if shuffle else dset
#     dset = dset.batch(bsize).prefetch(AUTO)

#     return dset

# # strategy = auto_select_accelerator()
# # BATCH_SIZE = strategy.num_replicas_in_sync * 16

# label_cols = ['negative', 'typical', 'indeterminate', 'atypical']
# study_df[label_cols] = 0

# test_decoder = build_decoder(with_labels=False,
#                              target_size=(STUDY_DIMS[0],
#                                           STUDY_DIMS[0]), ext='png')
# test_dataset = build_dataset(
#     test_paths, bsize=BATCH_SIZE, repeat=False, 
#     shuffle=False, augment=False, cache=False,
#     decode_fn=test_decoder
# )

# with tf.device('/device:GPU:0'):
#     models = []
#     models0 = tf.keras.models.load_model(f'{MODEL_PATH}/model0.h5',
#                                          custom_objects={'KerasLayer': KerasLayerWrapper})
#     models1 = tf.keras.models.load_model(f'{MODEL_PATH}/model1.h5',
#                                          custom_objects={'KerasLayer': KerasLayerWrapper})
#     models2 = tf.keras.models.load_model(f'{MODEL_PATH}/model2.h5',
#                                          custom_objects={'KerasLayer': KerasLayerWrapper})
#     models3 = tf.keras.models.load_model(f'{MODEL_PATH}/model3.h5',
#                                          custom_objects={'KerasLayer': KerasLayerWrapper})
#     models4 = tf.keras.models.load_model(f'{MODEL_PATH}/model4.h5',
#                                          custom_objects={'KerasLayer': KerasLayerWrapper})
#     models.append(models0)
#     models.append(models1)
#     models.append(models2)
#     models.append(models3)
#     models.append(models4)

# study_df[label_cols] = sum([model.predict(test_dataset, verbose=1) for model in models]) / len(models)
# study_df['PredictionString'] = study_df[label_cols].apply(lambda row: f'negative {row.negative} 0 0 1 1 typical {row.typical} 0 0 1 1 indeterminate {row.indeterminate} 0 0 1 1 atypical {row.atypical} 0 0 1 1', axis=1)

# del models
# del models0, models1, models2, models3, models4
# del test_dataset, test_decoder
# gc.collect()

## 2CLASS Predict

In [10]:
MODEL_PATH = '/kaggle/input/d/hyeonseop/siimeffnetv2keras2classtrain'

test_paths = image_df.image_path.tolist()
image_df['none'] = 0
label_cols = ['none']

test_decoder = build_decoder(with_labels=False,
                             target_size=(IMAGE_DIMS[0],
                                          IMAGE_DIMS[0]), ext='png')
test_dataset = build_dataset(
    test_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False,
    decode_fn=test_decoder
)

with tf.device('/device:GPU:0'):
    models = []
    models0 = tf.keras.models.load_model(f'{MODEL_PATH}/model0.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models1 = tf.keras.models.load_model(f'{MODEL_PATH}/model1.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models2 = tf.keras.models.load_model(f'{MODEL_PATH}/model2.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models3 = tf.keras.models.load_model(f'{MODEL_PATH}/model3.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models4 = tf.keras.models.load_model(f'{MODEL_PATH}/model4.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models.append(models0)
    models.append(models1)
    models.append(models2)
    models.append(models3)
    models.append(models4)

# image_df[label_cols] = np.argmax(sum([model.predict(test_dataset, verbose=1) for model in models]) / len(models) ,axis=1)
pred_2cls = sum([model.predict(test_dataset, verbose=1) for model in models]) / len(models)
image_df[label_cols] = np.max(pred_2cls , axis=1)

del models
del models0, models1, models2, models3, models4
del test_dataset, test_decoder
gc.collect()

#opacity: 0, none: 1

1/1 [==============================] - 3s 3s/step


4748081

## YOLO

In [11]:
import os

from PIL import Image
import pandas as pd
from tqdm.auto import tqdm

# .dcm to .png

In [12]:
if fast_sub:
    test_df = fast_df.copy()
#     test_df = test_df.iloc[:,:2]
#     test_df.columns = ['id', 'PredictionString']
else:
#     test_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
    test_df = image_df.copy()
    
test_df = test_df.reset_index(drop=True) 
test_df

,image_id,PredictionString,level,dcm_path,study_id
0,2a2f54eb48df_image,none 1 0 0 1 1,image,/kaggle/input/siim-covid19-detection/test/def2...,def217a20092_study
1,27d3da7a8e3d_image,none 1 0 0 1 1,image,/kaggle/input/siim-covid19-detection/test/e5eb...,e5eb6f1abf7f_study


In [13]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [14]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [15]:
# split = 'test'
# save_dir = f'/kaggle/tmp/{split}/'

# os.makedirs(save_dir, exist_ok=True)

# save_dir = f'/kaggle/tmp/{split}/study/'
# os.makedirs(save_dir, exist_ok=True)
# if fast_sub:
#     xray = read_xray('../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
#     im = resize(xray, size=600)  
#     study = '00086460a852' + '_study.png'
#     im.save(os.path.join(save_dir, study))
#     xray = read_xray('../input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
#     im = resize(xray, size=600)  
#     study = '000c9c05fd14' + '_study.png'
#     im.save(os.path.join(save_dir, study))
# else:   
#     for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
#         for file in filenames:
#             # set keep_ratio=True to have original aspect ratio
#             xray = read_xray(os.path.join(dirname, file))
#             im = resize(xray, size=600)  
#             study = dirname.split('/')[-2] + '_study.png'
#             im.save(os.path.join(save_dir, study))

In [16]:
# import shutil

# shutil.rmtree('/kaggle/tmp/test/image_detect')
# shutil.rmtree('runs/detect/exp/labels')

In [17]:
# os.mkdir('runs/detect/exp/labels')

In [18]:
image_id = []
dim0 = []
dim1 = []
splits = []

split = 'test'
save_dir = f'/kaggle/tmp/{split}/'

os.makedirs(save_dir, exist_ok=True)


save_dir = f'/kaggle/tmp/{split}/image_detect/'
os.makedirs(save_dir, exist_ok=True)
# if fast_sub:
for i in tqdm(range(len(test_df))):
#         xray = read_xray('../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
    xray = read_xray(test_df.loc[i,'dcm_path'])
    im = resize(xray, size=512)  
    im.save(os.path.join(save_dir, test_df.loc[i,'image_id'] + '.png'))
    image_id.append(test_df.loc[i,'image_id'].replace("_image",""))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)
#         xray = read_xray('../input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
#         im = resize(xray, size=512)  
#         im.save(os.path.join(save_dir, '51759b5579bc_image.png'))
#         image_id.append('51759b5579bc.dcm'.replace('.dcm', ''))
#         dim0.append(xray.shape[0])
#         dim1.append(xray.shape[1])
#         splits.append(split)
# else:
#     for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
#         for file in filenames:
#             # set keep_ratio=True to have original aspect ratio
#             xray = read_xray(os.path.join(dirname, file))
#             im = resize(xray, size=512)  
#             im.save(os.path.join(save_dir, file.replace('.dcm', '_image.png')))
#             image_id.append(file.replace('.dcm', ''))
#             dim0.append(xray.shape[0])
#             dim1.append(xray.shape[1])
#             splits.append(split)
meta = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits})

  0%|          | 0/2 [00:00<?, ?it/s]

# yolov5 predict

In [19]:
from numba import cuda
import torch
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

<weakproxy at 0x7fec6b765d70 to Device at 0x7fec6aabd490>

In [20]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
import torch

In [21]:
meta = meta[meta['split'] == 'test']
# if fast_sub:
test_df = test_df.iloc[:,:2]
test_df.columns = ['id', 'PredictionString']
# else:
#     test_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
# test_df = test_df.reset_index(drop=True) 
meta['image_id'] = meta['image_id'] + '_image'
meta.columns = ['id', 'dim0', 'dim1', 'split']
test_df = pd.merge(test_df, meta, on = 'id', how = 'left')
test_df

,id,PredictionString,dim0,dim1,split
0,2a2f54eb48df_image,none 1 0 0 1 1,2996,3052,test
1,27d3da7a8e3d_image,none 1 0 0 1 1,3480,4240,test


In [22]:
dim = 512 #1024, 256, 'original'
test_dir = f'/kaggle/tmp/{split}/image_detect'
weights_dir = '/kaggle/input/image-train/yolov5/runs/train/exp/weights/best.pt'

shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5') # install dependencies

import torch
#from IPython.display import Image, clear_output  # to display images

#clear_output()
#print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))


!python detect.py --weights $weights_dir\
--img 512\
--conf 0.001\
--iou 0.5\
--source $test_dir\
--save-txt --save-conf --exist-ok
def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]

    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height

    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

    return bboxes
image_ids = []
PredictionStrings = []

for file_path in tqdm(glob('runs/detect/exp/labels/*.txt')):
    image_id = file_path.split('/')[-1].split('.')[0]
    w, h = test_df.loc[test_df.id==image_id,['dim1', 'dim0']].values[0]
    f = open(file_path, 'r')
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
    bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 12).astype(str))
    for idx in range(len(bboxes)):
        bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
    image_ids.append(image_id)
    PredictionStrings.append(' '.join(bboxes))


pred_df = pd.DataFrame({'id':image_ids,
                        'PredictionString':PredictionStrings})

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.001, device='', exist_ok=True, img_size=512, iou_thres=0.5, name='exp', project='runs/detect', save_conf=True, save_txt=True, source='/kaggle/tmp/test/image_detect', update=False, view_img=False, weights=['/kaggle/input/image-train/yolov5/runs/train/exp/weights/best.pt'])
Fusing layers... 
image 1/2 /kaggle/tmp/test/image_detect/27d3da7a8e3d_image.png: 512x512 25 0. opacitys, Done. (0.059s)
image 2/2 /kaggle/tmp/test/image_detect/2a2f54eb48df_image.png: 512x512 9 0. opacitys, Done. (0.047s)
Results saved to runs/detect/exp
2 labels saved to runs/detect/exp/labels
Done. (0.342s)


  0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
test_df = test_df.drop(['PredictionString'], axis=1)
sub_df = pd.merge(test_df, pred_df, on = 'id', how = 'left').fillna("none 1 0 0 1 1")
sub_df = sub_df[['id', 'PredictionString']]
for i in range(sub_df.shape[0]):
    if sub_df.loc[i,'PredictionString'] == "none 1 0 0 1 1":
        continue
    sub_df_split = sub_df.loc[i,'PredictionString'].split()
    sub_df_list = []
    for j in range(int(len(sub_df_split) / 6)):
        sub_df_list.append('opacity')
        sub_df_list.append(sub_df_split[6 * j + 1])
        sub_df_list.append(sub_df_split[6 * j + 2])
        sub_df_list.append(sub_df_split[6 * j + 3])
        sub_df_list.append(sub_df_split[6 * j + 4])
        sub_df_list.append(sub_df_split[6 * j + 5])
    sub_df.loc[i,'PredictionString'] = ' '.join(sub_df_list)
# sub_df['none'] = df_2class['none'] 
image_df = image_df['none'].reset_index()
sub_df['none'] = image_df['none'] 
for i in range(sub_df.shape[0]):
    if sub_df.loc[i,'PredictionString'] != 'none 1 0 0 1 1':
        sub_df.loc[i,'PredictionString'] = sub_df.loc[i,'PredictionString'] + ' none ' + str(sub_df.loc[i,'none']) + ' 0 0 1 1'
sub_df = sub_df[['id', 'PredictionString']]   
# df_study = study_df.loc[:study_len]
df_study = study_df.loc[:,['study_id', 'PredictionString']]
df_study.columns = ['id', 'PredictionString']
df_study = df_study.append(sub_df).reset_index(drop=True)
df_study.to_csv('/kaggle/working/submission.csv',index = False)  
shutil.rmtree('/kaggle/working/yolov5')

In [24]:
sub_df

,id,PredictionString
0,2a2f54eb48df_image,opacity 0.001171109965 2498 1644 2861 2142 opa...
1,27d3da7a8e3d_image,opacity 0.001051899977 398 2059 1615 2923 opac...


In [25]:
test_df

,id,dim0,dim1,split
0,2a2f54eb48df_image,2996,3052,test
1,27d3da7a8e3d_image,3480,4240,test


In [26]:
pred_df

,id,PredictionString
0,2a2f54eb48df_image,0 0.001171109965 2498 1644 2861 2142 0 0.00124...
1,27d3da7a8e3d_image,0 0.001051899977 398 2059 1615 2923 0 0.001165...


In [27]:
image_df

,index,none
0,351,0.856867
1,1206,0.570324


In [28]:
study_df

,study_id,PredictionString,level,dcm_path,image_id,image_path,negative,typical,indeterminate,atypical
1046,ce2d48a26e5e_study,negative 0.08432073891162872 0 0 1 1 typical 0...,study,/kaggle/input/siim-covid19-detection/test/ce2d...,a1883aeaa8f0_image,/kaggle/tmp/test/study/ce2d48a26e5e_study.png,0.084321,0.329937,0.237947,0.347795
68,0ec0901a5d45_study,negative 0.1501372754573822 0 0 1 1 typical 0....,study,/kaggle/input/siim-covid19-detection/test/0ec0...,4d7b06512cc1_image,/kaggle/tmp/test/study/0ec0901a5d45_study.png,0.150137,0.358529,0.306055,0.185279


In [29]:
df_study

,id,PredictionString
0,ce2d48a26e5e_study,negative 0.08432073891162872 0 0 1 1 typical 0...
1,0ec0901a5d45_study,negative 0.1501372754573822 0 0 1 1 typical 0....
2,2a2f54eb48df_image,opacity 0.001171109965 2498 1644 2861 2142 opa...
3,27d3da7a8e3d_image,opacity 0.001051899977 398 2059 1615 2923 opac...
